# Graph classification vs Node classification

- Graph classification is very similar to a general image classification in CNN. Graph classification refers to the problem of classifiying entire graphs (in contrast to nodes), given a dataset of graphs, based on some structural graph properties. 
- In node classification, we use embedding vector for each node to come up with x_feature for each node. Here, for graph classification, we want to embed entire graphs, and we want to embed those graphs in such a way so that they are linearly separable given a task at hand.


<img src="./img/graph_classification.png" height="400" width="400"/>


In [9]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset

import matplotlib.pyplot as plt

# AIDS dataset graph

- As you will see below, we will have 2000 graph, each graph is represented by XX number of nodes (different nodes in each graph, similar to NLP where we have sentences with different lengths and we used padding to make all sentences at the same length). But each node is represented by an embedding of size 38 (similar to NLP where each word was embedded). Each graph has an attribute indicating if the compound is active or inactive against HIV (basically the two classes we want to figure out).

- There are two classes for all these graphs: positive and negative AIDS molecular structure
- TUDataset library from Pytorch Geometric: A collection of benchmark datasets for learning with graphs


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

dataset_name = "AIDS"
dataset = TUDataset(".", name=dataset_name)

In [12]:
print()
print('Dataset: {}'.format(dataset))
print('====================')
print('Number of graphs: {}'.format(len(dataset)))
print('Size of x_features: {}'.format(dataset.num_features))
print('Number of classes: {}'.format(dataset.num_classes))

print("one sample out of 2000 AIDS graph data: ", dataset[0])
print("one sample out of 2000 AIDS graph data: ", dataset[1])

print("\n")
print('=============================================================')
print('LET\'s EXPLORE ONE OF THESE 2000 GRAPHS')


data = dataset[0]
print()
print(data)
print('-------------------------------------------------------------')

# Gather some statistics about the first graph.
print('Number of nodes: {}'.format(data.num_nodes))
print('Number of edges: {}'.format(data.num_edges))
print('Average node degree: {:.2f}'.format(data.num_edges / data.num_nodes))
print("number of features (size of embedding vector ): ",data.num_features)
print('Has isolated nodes: {}'.format(data.has_isolated_nodes()))
print('Has self-loops: {}'.format(data.has_self_loops()))
print('Is undirected: {}'.format(data.is_undirected()))


Dataset: AIDS(2000)
Number of graphs: 2000
Size of x_features: 38
Number of classes: 2
one sample out of 2000 AIDS graph data:  Data(edge_index=[2, 106], x=[47, 38], edge_attr=[106, 3], y=[1])
one sample out of 2000 AIDS graph data:  Data(edge_index=[2, 22], x=[11, 38], edge_attr=[22, 3], y=[1])


LET's EXPLORE ONE OF THESE 2000 GRAPHS

Data(edge_index=[2, 106], x=[47, 38], edge_attr=[106, 3], y=[1])
-------------------------------------------------------------
Number of nodes: 47
Number of edges: 106
Average node degree: 2.26
number of features (size of embedding vector ):  38
Has isolated nodes: False
Has self-loops: False
Is undirected: True


As seen above, this dataset provides 2000 different graphs, and the task is to classify each graph into one out of two classes.

By inspecting the first graph object of the dataset, we can see that it comes with 47 nodes (with 38-dimensional feature vectors) and 106 edges (leading to an average node degree of 2.26). It also comes with exactly one graph label (y=[1]), and, in addition to previous datasets, provides addtional 3-dimensional edge features (edge_attr=[106, 3]). However, for the sake of simplicity, we will not make use of those x_feature and only focus on x_feature comes from node embedding.

**PyTorch Geometric** provides some useful utilities for working with graph datasets, e.g., we can shuffle the dataset and use the first 1500 graphs as training graphs, while using the remaining ones for testing:

In [14]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:1500]
test_dataset = dataset[1500:]

print('Number of training graphs: {}'.format(len(train_dataset)) )
print('Number of test graphs: {}'.format(len(test_dataset)) )

Number of training graphs: 1500
Number of test graphs: 500


# Mini-batching of graphs (DataLoader Class)

Since graphs in graph classification datasets are usually small, a good idea is to **batch the graphs** before inputting them into a Graph Neural Network to guarantee full GPU utilization.
In the image or language domain, this procedure is typically achieved by **rescaling** or **padding** each example into a set of equally-sized shapes, and examples are then grouped in an additional dimension.
The length of this dimension is then equal to the number of examples grouped in a mini-batch and is typically referred to as the `batch_size`.

However, for GNNs the two approaches described above are either not feasible or may result in a lot of unnecessary memory consumption.
Therefore, PyTorch Geometric opts for another approach to achieve parallelization across a number of examples. Here, adjacency matrices are stacked in a diagonal fashion (creating a giant graph that holds multiple isolated subgraphs), and node and target features are simply concatenated in the node dimension:

<img src="./img/graph_mini_batching.png" height="400" width="400"/>


This procedure has some crucial advantages over other batching procedures:

GNN operators that rely on a message passing scheme do not need to be modified since messages are not exchanged between two nodes that belong to different graphs.

There is no computational or memory overhead since adjacency matrices are saved in a sparse fashion holding only non-zero entries, i.e., the edges.

PyTorch Geometric automatically takes care of batching multiple graphs into a single giant graph with the help of the torch_geometric.data.DataLoader class.


Below, for test dataset, we opt for a batch_size of 64, leading to 8 (randomly shuffled) mini-batches, containing all  2⋅64+22=150  graphs. Furthermore, each Batch object is equipped with a batch vector, which maps each node to its respective graph in the batch:

$$batch = [0, ...,0,1, ..., 1, 2]$$

In [17]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(test_loader):
    print('Step {}'.format(step + 1))
    print('=======')
    print('Number of graphs in the current batch: {}'.format(data.num_graphs) )
    print(data)
    print()

Step 1
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2024], x=[993, 38], edge_attr=[2024, 3], y=[64], batch=[993], ptr=[65])

Step 2
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2802], x=[1343, 38], edge_attr=[2802, 3], y=[64], batch=[1343], ptr=[65])

Step 3
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2022], x=[968, 38], edge_attr=[2022, 3], y=[64], batch=[968], ptr=[65])

Step 4
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 1890], x=[908, 38], edge_attr=[1890, 3], y=[64], batch=[908], ptr=[65])

Step 5
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 1918], x=[945, 38], edge_attr=[1918, 3], y=[64], batch=[945], ptr=[65])

Step 6
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 1950], x=[950, 38], edge_attr=[1950, 3], y=[64], batch=[950], ptr=[65])

Step 7
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2252], x=[1083, 38], edge_attr=[2252, 3], 

If we look at the report above and focus on the 1st batch, we will see that 64 graphs are bacthed together as if they are in one big graph (that all 64 sub-graphs dis-connected). The fact the size of x if [993, 38], the 993 shows number of total nodes in all these 64 dis-connected graphs.

# Training a Graph Neural Network (GNN)

Training a GNN for graph classification usually follows a simple recipe:

    - Embed each node by performing multiple rounds of message passing
    - Aggregate node embeddings into a unified graph embedding (readout layer)
    - Train a final classifier on the graph embedding
    
There exists multiple **message passing** in literature, but the most common one and simplest is to simply take the average of all neighboring node and the node itself (thier embeddings):

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

Above, basically is concat all the embedding vectors of neighbouring nodes and take the average of them. It is common practice for **GCNConv**. Alternatively, below, we could use **GraphConv** where 

$$
\mathbf{x}_v^{(\ell+1)} = \mathbf{W}^{(\ell + 1)}_1 \mathbf{x}_v^{(\ell)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{w \in \mathcal{N}(v)} \mathbf{x}_w^{(\ell)}
$$

Where we do not simply average the node iteself and all the neighbors. We add up all the neighbors (with some Weight factor) and then add up that vector with the node embedding.

PyTorch Geometric provides this functionality via [`torch_geometric.nn.global_mean_pool`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_mean_pool), which takes in the node embeddings of all nodes in the mini-batch and the assignment vector `batch` to compute a graph embedding of size `[batch_size, hidden_channels]` for each graph in the batch.

The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training.

Below, we make use of the GCNConv with  ReLU(x)=max(x,0)  activation for obtaining localized node embeddings, before we apply our final classifier on top of a graph readout layer.

Below shows how the GCNConv does the message passing. Ir inherites from The “MessagePassing” Base Class:
https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html


In [19]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

feature_size = dataset.num_node_features # here in this exampe x_feature is size of 38
num_classes = dataset.num_classes


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(feature_size, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(38, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


#### Traning step
Let's train our network for a few epochs to see how well it performs on the training as well as test set:

In [20]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.7980, Test Acc: 0.8060
Epoch: 002, Train Acc: 0.7980, Test Acc: 0.8060
Epoch: 003, Train Acc: 0.7980, Test Acc: 0.8060
Epoch: 004, Train Acc: 0.7980, Test Acc: 0.8060
Epoch: 005, Train Acc: 0.7953, Test Acc: 0.8020
Epoch: 006, Train Acc: 0.7987, Test Acc: 0.8040
Epoch: 007, Train Acc: 0.7907, Test Acc: 0.7900
Epoch: 008, Train Acc: 0.7980, Test Acc: 0.8040
Epoch: 009, Train Acc: 0.7900, Test Acc: 0.7580
Epoch: 010, Train Acc: 0.7787, Test Acc: 0.7680
Epoch: 011, Train Acc: 0.7967, Test Acc: 0.7920
Epoch: 012, Train Acc: 0.7967, Test Acc: 0.7920
Epoch: 013, Train Acc: 0.7987, Test Acc: 0.7860
Epoch: 014, Train Acc: 0.8047, Test Acc: 0.7820
Epoch: 015, Train Acc: 0.7920, Test Acc: 0.7880
Epoch: 016, Train Acc: 0.8120, Test Acc: 0.7940
Epoch: 017, Train Acc: 0.7953, Test Acc: 0.7620
Epoch: 018, Train Acc: 0.8233, Test Acc: 0.7920
Epoch: 019, Train Acc: 0.8027, Test Acc: 0.7900
Epoch: 020, Train Acc: 0.8233, Test Acc: 0.7940
Epoch: 021, Train Acc: 0.8267, Test Acc:

As mentioned above, in **GraphConv** we do the message passing a little different than the **GCNConv** layer:

$$
\mathbf{x}_v^{(\ell+1)} = \mathbf{W}^{(\ell + 1)}_1 \mathbf{x}_v^{(\ell)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{w \in \mathcal{N}(v)} \mathbf{x}_w^{(\ell)}
$$

In [21]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch_geometric.nn import global_mean_pool

feature_size = dataset.num_node_features # here in this exampe x_feature is size of 38
num_classes = dataset.num_classes


class GCN_graph_conv(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN_graph_conv, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(feature_size, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN_graph_conv(hidden_channels=64)
print(model)

GCN_graph_conv(
  (conv1): GraphConv(38, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [22]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN_graph_conv(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.7773, Test Acc: 0.7740
Epoch: 002, Train Acc: 0.7987, Test Acc: 0.8140
Epoch: 003, Train Acc: 0.7980, Test Acc: 0.7920
Epoch: 004, Train Acc: 0.8120, Test Acc: 0.8180
Epoch: 005, Train Acc: 0.8060, Test Acc: 0.8080
Epoch: 006, Train Acc: 0.8413, Test Acc: 0.8520
Epoch: 007, Train Acc: 0.8747, Test Acc: 0.8240
Epoch: 008, Train Acc: 0.8573, Test Acc: 0.8560
Epoch: 009, Train Acc: 0.8807, Test Acc: 0.8560
Epoch: 010, Train Acc: 0.9007, Test Acc: 0.8760
Epoch: 011, Train Acc: 0.9020, Test Acc: 0.8640
Epoch: 012, Train Acc: 0.8973, Test Acc: 0.8820
Epoch: 013, Train Acc: 0.8940, Test Acc: 0.8740
Epoch: 014, Train Acc: 0.9140, Test Acc: 0.9040
Epoch: 015, Train Acc: 0.9207, Test Acc: 0.9000
Epoch: 016, Train Acc: 0.9253, Test Acc: 0.8960
Epoch: 017, Train Acc: 0.9253, Test Acc: 0.8820
Epoch: 018, Train Acc: 0.9213, Test Acc: 0.8920
Epoch: 019, Train Acc: 0.9153, Test Acc: 0.8900
Epoch: 020, Train Acc: 0.9300, Test Acc: 0.8920
Epoch: 021, Train Acc: 0.8960, Test Acc:

# Reference:
https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=HvhgQoO8Svw4